In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import copy
import datetime
from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
df = pd.read_csv("clean_data.csv", index_col=0)
df

# Correlation distribution

In [ ]:
plt.figure(figsize=(10,3))
plt.axhline(0)
for i in [-0.15, -0.1, -0.05, 0.05, 0.1, 0.15]:
    plt.axhline(i, color = "black", linestyle=':', linewidth = 1)

plt.xlabel("Predictor")
plt.ylabel("Corr(Predictor, score)")
bars = bars = df.corr()['score'][df.corr()['score'].abs().sort_values(ascending=False).index][range(1,11)].plot(kind='bar')
plt.savefig('corr_dist.png', dpi=300, bbox_inches="tight")

# MLR

In [ ]:
def find_predictors(df):
    
    # Sort by absolute correlation
    predictors = abs(df.corr()['score']).sort_values(ascending=False).drop("score")

    r2_max = 0
    i_max = 0

    # add variables until model is overfitting
    for i, col in enumerate(list(predictors.index)):
        X = df[predictors.index[:i+1]]
        X_train, X_test, y_train, y_test = train_test_split(X, df["score"], test_size=0.3, random_state=42)
        reg = LinearRegression().fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        if r2 > r2_max:
            r2_max = r2
            i_max = i+1

    # return predictors
    return predictors[0:i_max] 


In [ ]:
# train again best model
predictors  = find_predictors(df)
X_train, X_test, y_train, y_test = train_test_split(df[list(predictors.index)], df["score"], test_size=0.3, random_state=42)
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
r2 = r2_score(y_test, y_pred)
explained_var = explained_variance_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)

len(predictors), round(r2, 5), round(MSE, 5), round(explained_var,5)

## Check correlation between independent variables

In [ ]:
corr = df.iloc[:,0:-1].corr()
high_corr1, high_corr2 = np.where(np.abs(corr)>0.5)
# get indicics of highly correlated features in feature matrix
A = []
for i in range(len(high_corr1)):
    if high_corr1[i]!=high_corr2[i]:
        A.append((high_corr1[i],high_corr2[i]))
        
# transform indices to names of columns, add to tuple together with correlation of the features
B = []
for i in range(len(A)):
    B.append((df.columns[A[i][0]],df.columns[A[i][1]], round(corr.iloc[A[i][0],A[i][1]],3)))
    
# transform indices to names of columns, add to tuple together with correlation of the features
B = []
for i in range(len(A)):
    B.append((df.columns[A[i][0]],df.columns[A[i][1]], round(corr.iloc[A[i][0],A[i][1]],3)))
B
    

## Histograms conditioned on MMO, Indie and release_year

In [ ]:
bins = np.linspace(0,1,21)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3,figsize=(16, 5))
fig.suptitle('MMO', fontsize=16)
ax1.hist(df.loc[df['MMO'] == 1]["score"],bins)
ax2.hist(df.loc[df["MMO"] == 0]["score"],bins)
ax1.set_title("MMO=1")
ax2.set_title("MMO=0")
ax3.hist(df.loc[df['MMO'] == 1]["score"],bins, density=True, alpha=0.5)
ax3.hist(df.loc[df["MMO"] == 0]["score"],bins, density=True, alpha=0.5)

confirm with Whitney U-Test:

In [ ]:
stats.mannwhitneyu(df.loc[df["MMO"] == 0]["score"], df.loc[df['MMO'] == 1]["score"],alternative="less")

whitneyu provides evidence that distribution of MMO=1 is NOT higher than distribution of MMO=0.

In [ ]:
bins = np.linspace(0,1,21)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3,figsize=(16, 5))
fig.suptitle('Downloadable Content', fontsize=16)
ax1.hist(df.loc[df['Downloadable Content'] == 1]["score"],bins)
ax2.hist(df.loc[df['Downloadable Content'] == 0]["score"],bins)
ax1.set_title("Downloadable Content=1")
ax2.set_title("Downloadable Content=0")
ax3.hist(df.loc[df['Downloadable Content'] == 1]["score"],bins, density=True, alpha=0.5)
ax3.hist(df.loc[df['Downloadable Content'] == 0]["score"],bins, density=True, alpha=0.5)

In [ ]:
bins = np.linspace(0,1,21)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3,figsize=(16, 5))
fig.suptitle('Downloadable Content', fontsize=16)
ax1.hist(df.loc[df['Indie'] == 1]["score"],bins)
ax2.hist(df.loc[df['Indie'] == 0]["score"],bins)
ax1.set_title('Indie=1')
ax2.set_title('Indie=0')
ax3.hist(df.loc[df['Indie'] == 1]["score"],bins, density=True, alpha=0.5)
ax3.hist(df.loc[df['Indie'] == 0]["score"],bins, density=True, alpha=0.5)

In [ ]:
stats.mannwhitneyu(df.loc[df['Indie'] == 0]["score"], df.loc[df['Indie'] == 1]["score"], alternative="less")

Whitneyu provides evidence that Indie games are better than non-indie games

In [ ]:
A = np.zeros(len(set(df["relase_year"])))
for i in range(len(set(df["relase_year"]))):
    A[i] = np.mean(df.loc[lambda df: df.relase_year == 1998+i,"score"])
    

In [ ]:
plt.plot(A,".")
plt.ylabel("Score")
plt.xlabel("release year")
plt.xticks([2,7,12,17,22],labels=["2000","2005", "2010", "2015", "2020"])
plt.title("average game-score by release year")

From the looks of it, games seem to have been better (rated) in the past. However, the plot is to be enjoyed with caution,
e.g. for the years 1998,1999 are only 1-2 games in the data set.

# NN approach

In [ ]:
# standardize data before using it as NN input
df_stand = df.copy(deep=True)
age = df["required_age"]
df_stand["required_age"] = (age-np.mean(age))/np.std(age)
price = np.array(df["price"]).reshape(-1)
df_stand["price"] = (price-np.mean(price))/np.std(price)
year = df["relase_year"]
year = df["relase_year"]
df_stand["relase_year"] = (year-np.mean(year))/np.std(year)

In [ ]:
device = (torch.device('cuda') if torch.cuda.is_available()
else torch.device('cpu'))

In [ ]:
class SteamData(Dataset):
    def __init__(self, df):
        super().__init__()
        self.df = df
        self.data_size = len(self.df)
        self.num_variables = len(self.df.columns[0:-1])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        IVs = torch.from_numpy(np.array(self.df.iloc[idx,0:-1],dtype=np.double))
        target = torch.from_numpy(np.array(self.df.loc[idx,"score"], dtype=np.double))
        
        sample = (IVs,target)
        return sample

In [ ]:
Dataloader_steam = SteamData(df_stand)

BATCHSIZE = 64

generator=torch.Generator().manual_seed(42)

train_size = int(0.7 * len(df_stand))
test_size = len(df_stand) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(Dataloader_steam, [train_size, test_size], generator =generator)

SteamData_train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCHSIZE,
shuffle=True)

SteamData_val_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCHSIZE,
shuffle=False)

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(53,128)
        self.lin2 = nn.Linear(128,256)
        self.lin3 = nn.Linear(256,128)
        self.lin4 = nn.Linear(128,1)
        
    def forward(self,x):
        out = F.leaky_relu(self.lin1(x))
        out = F.leaky_relu(self.lin2(out))
        out = F.leaky_relu(self.lin3(out))
        out = torch.sigmoid(self.lin4(out))
        return out

In [ ]:
model = NN()
model = model.to(device=device)
model = model.double()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()
model.train()
n_epochs = 20

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):
    loss_train = np.array(n_epochs)
    loss_val = np.array(n_epochs)
    for epoch in range(1,n_epochs+1):
        model.train()
        total_loss_train = 0
        total_loss_val = 0
        for IV, score in train_loader:
            IV = IV.to(device=device)
            score = score.to(device=device)
            outputs = model(IV)
            loss_train = loss_fn(outputs.flatten(), score)
            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()
            
            total_loss_train+=loss_train
        
        for IV, score in val_loader:
            model.eval()
            with torch.no_grad():
                IV = IV.to(device=device)
                score = score.to(device=device)
                outputs = model(IV)
                loss_val = loss_fn(outputs.flatten(), score)
                
                total_loss_val+=loss_val
       
            
            

        if epoch ==1 or epoch % 1 == 0:
            print("{} Epoch {}, Training loss {}, Validation Loss: {}".format(
            datetime.datetime.now(), epoch,
            total_loss_train/ len(train_loader), total_loss_val/len(val_loader)))

In [ ]:
training_loop(n_epochs, optimizer, model, loss_fn, SteamData_train_loader, SteamData_val_loader)

## Check testset with NN model

In [ ]:
test_samples = np.zeros((len(SteamData_val_loader.dataset),53))
test_samples_score = np.zeros(len(SteamData_val_loader.dataset))
idx=0
for IV,score in SteamData_val_loader:
    test_samples[idx:idx+IV.shape[0]] = IV
    test_samples_score[idx:idx+IV.shape[0]] = score
    idx=idx+IV.shape[0]
test_samples = (torch.from_numpy(test_samples)).to(device)
with torch.no_grad():
    y_pred_NN = model(test_samples)

explained_variance_score(test_samples_score,y_pred_NN.cpu().numpy())

## Difference between regression and NN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[df.columns[best_fit_indices_min[1]]],df["score"] , test_size=0.3, random_state=42)
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
explained_var = explained_variance_score(y_test, y_pred)
MSE = mean_squared_error(y_pred, y_test)
explained_var, MSE

In [ ]:
bins = np.linspace(0.4,1,21)
bins2 = np.linspace(-0.5,0.75,30)
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(16, 5))
ax1.hist(y_pred, bins, alpha=0.5, label="linear regression")
ax1.hist(y_pred_NN.cpu().numpy().reshape(-1), bins, alpha=0.5, label="NN")
ax1.legend()
ax1.set_title("predicted scores")
ax1.set_xlabel("score")
ax2.hist(y_pred-y_test, bins2, alpha=0.5, label="linear regression")
ax2.hist(y_pred_NN.cpu().numpy().reshape(-1)- test_samples_score, bins2, alpha=0.5, label="NN")
ax2.legend()
ax2.set_title("residuals of predictions")
ax2.set_xlabel("residuals/erros")